# Scrape Tickers, 10K reports merging, Clean GICS classification, List of CIKs for relevant tickers, Scrape Wikipedia data, Clean Russell Ratios

- [test](#TEST-sdf)

In [1]:
import os
from lxml import etree
import bs4 as bs
import requests
import time
import sys
from selenium import webdriver
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm # If ur using Jupyter Lab
from tqdm.notebook import tqdm # If you are using Jupyter Notebook
import wikipedia as wiki
import re
import yfinance as yf

In [2]:
pd.set_option('display.max_rows', 100)

# Scrape Tickers

- All non-scrapy scrappers are here
- This section is just to get a sets of tickers (the other info collected are not that necessary)

### SnP500

In [ ]:
snp_tickers_df = pd.read_html('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies', match= 'GICS')[0]
snp_tickers_df = snp_tickers_df[['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'CIK']]

In [ ]:
snp_tickers_df.columns = ['Ticker', 'Name', 'Sector', 'Sub Industry', 'CIK']
snp_tickers_df.Ticker = snp_tickers_df.Ticker.str.replace('.', '-')
snp_tickers_df.head()

In [ ]:
# if not df not saved yet, do this
# snp_tickers_df.to_csv('data_out/snp_tickers_df.csv', index=False)
# snp_tickers_df.to_csv('scrapy_spiders/data_in/snp_tickers_df.csv', index=False)

In [4]:
snp_tickers_df = pd.read_csv('data_out/snp_tickers_df.csv')
snp_tickers_df.head()

,Ticker,Name,Sector,Sub Industry,CIK
0,MMM,3M Company,Industrials,Industrial Conglomerates,66740
1,ABT,Abbott Laboratories,Health Care,Health Care Equipment,1800
2,ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,1551152
3,ABMD,ABIOMED Inc,Health Care,Health Care Equipment,815094
4,ACN,Accenture plc,Information Technology,IT Consulting & Other Services,1467373


In [5]:
snp_tickers_df.shape

(505, 5)

In [ ]:
'''
Long winded method using BS4

def get_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        if ticker[-1:]=='\n':
            ticker=ticker[:-1]
        ticker=ticker.replace('.','')
        tickers.append(ticker)
    return tickers
'''
pass

# TEST sdf

### Russell 3000

- could not find reliable source with tickers and companies

##### Source 1

 - This is the best source for this project as it has 3000 tickers unlike the source below and all these tickers have GICS mappings

In [ ]:
russell_tickers_df = pd.read_csv('data_in/ticker_to_gics.csv', names=['Ticker', 'GICS'])
russell_tickers_df.Ticker = russell_tickers_df.Ticker.str.replace('/','-')
russell_tickers_df.drop('GICS', axis=1, inplace=True)
print(russell_tickers_df.shape)
russell_tickers_df.head()

In [ ]:
def get_ticker_fullname(ticker):
    try:
        cpy = yf.Ticker(ticker)
        name = cpy.info['longName']
        return name
    except:
        return None

In [ ]:
russell_tickers_df['Name'] = [get_ticker_fullname(ticker) for ticker in tqdm(russell_tickers_df.Ticker)]
russell_tickers_df

In [ ]:
# if not df not saved yet, do this
# russell_tickers_df.to_csv('data_out/russell_tickers_df.csv', index=False)
# russell_tickers_df.to_csv('scrapy_spiders/data_in/russell_tickers_df.csv', index=False)

In [32]:
russell_tickers_df = pd.read_csv('data_out/russell_tickers_df.csv')
print(russell_tickers_df.shape)
russell_tickers_df.head()

(2964, 2)


,Ticker,Name
0,A,"Agilent Technologies, Inc."
1,AA,Alcoa Corporation
2,AAL,American Airlines Group Inc.
3,AAN,"Aaron's Holdings Company, Inc."
4,AAOI,"Applied Optoelectronics, Inc."


##### Source 2

 - from a 3rd party source
 - only source which contains industry and sector

In [ ]:
russell_tickers_df = pd.read_html('http://www.kibot.com/Historical_Data/Russell_3000_Historical_Intraday_Data.aspx')[1]
russell_tickers_df = russell_tickers_df.rename(columns=russell_tickers_df.iloc[0]).drop(0)[
                        ['Symbol', 'Description', 'Industry', 'Sector']].reset_index(drop=True)

In [ ]:
russell_tickers_df.columns = ['Ticker', 'Name', 'Industry', 'Sector']
russell_tickers_df.Ticker = russell_tickers_df.Ticker.str.replace('.', '-')
russell_tickers_df.head()

In [ ]:
russell_tickers_df.shape

In [ ]:
len([i for i in snp_tickers_df.Ticker if i not in russell_tickers_df.Ticker.values])

=> SnP is not a subset of Russell

##### Source 3
- problem with this method is need to get tickers

In [ ]:
import tabula

In [ ]:
all_ticker_tables_df_list = tabula.read_pdf('data_in/russell3000.pdf', pages="all")

In [ ]:
russell_tickers_df2 = pd.concat(all_ticker_tables_df_list).reset_index(drop=True)
russell_tickers_df2.head()

In [ ]:
russell_tickers_df2[russell_tickers_df2.Ticker.str.contains(".", regex=False)]

### STI

In [ ]:
sti_ticker_df = pd.read_html('https://en.wikipedia.org/wiki/Straits_Times_Index', match= 'Stock Symbol')[0]

In [ ]:
sti_ticker_df.head()

In [ ]:
sti_ticker_df.to_csv('data_out/sti_ticker_df.csv')

# Yahoo Description, Price, Ratios scraping

- can try selenium grid for multiprocessing

In [4]:
tickers_df = russell_tickers_df

### Selenium

In [ ]:
import time
from selenium import webdriver
from multiprocessing import Pool
from IPython.display import display, HTML

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

##### Desc Scraping

In [ ]:
# %%time
ticker_list = tickers_df.Ticker[:2]
# ticker_list = ['GOOG', 'GOOGL']
ticker_desc_df = pd.DataFrame(columns = ['Ticker', 'Description', 'Sector', 'Industry'])
wrong_ticker_list = []

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--headless')
options.add_argument('--no-sandbox') # Bypass OS security model

# If you print, tqdm bar will not work
for ticker in tqdm(ticker_list):
    url = 'https://finance.yahoo.com/quote/'+ticker+'/profile?p='+ticker
    print(url)    
    
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    
    desc_xpath = '//*[@id="Col1-0-Profile-Proxy"]/section/section[2]/p'
    element_present = EC.presence_of_element_located(
                (By.XPATH, desc_xpath))
    try:
        WebDriverWait(driver, 10).until(element_present)
    except:
        print(ticker)
        wrong_ticker_list.append(ticker)
        continue
    
    desc = driver.find_element_by_xpath(desc_xpath).text
    sector = driver.find_element_by_xpath('//*[@id="Col1-0-Profile-Proxy"]/section/div[1]/div/div/p[2]/span[2]').text
    industry = driver.find_element_by_xpath('//*[@id="Col1-0-Profile-Proxy"]/section/div[1]/div/div/p[2]/span[4]').text
    
    ticker_desc_df.loc[len(ticker_desc_df)] = [ticker, desc, sector, industry]
    driver.close() #need?
display(ticker_desc_df)
wrong_ticker_list

In [ ]:
ticker_desc_df.to_csv('data_out/russell_desc_yahoo_df.csv', index=False, encoding='utf-8-sig')

Comparison to Scrapy
- Sometime when running scrapy spider, it causes some links to stop be able to be accessed by this computer, which is a odd phenomenon
- after a while it is okay

Timing
- Scrapy: 78s
- Selenium: 1hr ++
    - Running headless makes it about 30% faster
    - Rruns faster than the original code (profs code) as well

##### Desc Scraping (using yfinance, faster)

In [20]:
def get_ticker_details(ticker):
    try:
        cpy = yf.Ticker(ticker)
        info = cpy.info
        name = info['longBusinessSummary']
        sector = info['sector']
        industry = info['industry']
        return [name, sector, industry]
    except:
        return [None] * 3

In [29]:
ticker_desc_df = tickers_df.copy()
ticker_desc_df[['Description', 'Sector', 'Industry']] = [get_ticker_details(ticker) for ticker in tqdm(ticker_desc_df.Ticker)]

In [33]:
ticker_desc_df = ticker_desc_df[['Ticker', 'Description', 'Sector', 'Industry']]
ticker_desc_df

,Ticker,Description,Sector,Industry
0,A,"Agilent Technologies, Inc. provides applicatio...",Healthcare,Diagnostics & Research
1,AA,"Alcoa Corporation, together with its subsidiar...",Basic Materials,Aluminum
2,AAL,"American Airlines Group Inc., through its subs...",Industrials,Airlines
3,AAN,"Aaron's Holdings Company, Inc., through its su...",Industrials,Rental & Leasing Services
4,AAOI,"Applied Optoelectronics, Inc. designs, manufac...",Technology,Semiconductors
...,...,...,...,...
2959,ZTS,"Zoetis Inc. discovers, develops, manufactures,...",Healthcare,Drug Manufacturers—Specialty & Generic
2960,ZUMZ,"Zumiez Inc., together with its subsidiaries, o...",Consumer Cyclical,Apparel Retail
2961,ZUO,"Zuora, Inc. provides cloud-based software on a...",Technology,Software—Infrastructure
2962,ZYNE,"Zynerba Pharmaceuticals, Inc. operates as a cl...",Healthcare,Drug Manufacturers—Specialty & Generic


In [34]:
ticker_desc_df.to_csv('data_out/russell_desc_yahoo_df.csv', index=False, encoding='utf-8-sig')

##### Try using multithreading with Selenium

In [ ]:
'''%%time

import multithreading_test

ticker_list = tickers_df.Ticker.head(5)
ticker_desc = pd.DataFrame(columns = ['Ticker', 'Description', 'Sector', 'Industry'])

p = Pool(processes = 4)
results = p.map_async(multithreading_test.get_ticker_desc_3, ticker_list)
output = results.get()

ticker_desc_df = pd.DataFrame(output, columns = ['Ticker', 'Description', 'Sector', 'Industry'])
display(ticker_desc_df)
'''
pass

##### Price Scraping (Method 1 : by clicking Downloads)

- Have to use selenium (and not scrapy) for price scraping as you need to interact with the webpage to extract the prices (i.e. clicking button or scrolling)
    - even if use pd.read_HTML(), it only outputs 100 elements (no interactivity)

In [ ]:
ticker_list = tickers_df.Ticker
DOWNLOAD_FOLDER = os.getcwd() + os.path.sep + 'data_out\\russell_price_csv_files\\'
wrong_ticker_list = []

date1 = datetime.strptime('20190101', "%Y%m%d")
date2 = datetime.strptime('20200101', "%Y%m%d")

time_str1 = str(int(datetime.timestamp(date1)))
time_str2 = str(int(datetime.timestamp(date2)))

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
prefs = {
            "profile.default_content_settings.popups": 0,
            "download.default_directory": DOWNLOAD_FOLDER,
            "directory_upgrade": True
        }
options.add_experimental_option('prefs', prefs)
options.add_argument('--headless')
options.add_argument('--no-sandbox') # Bypass OS security model

for ticker in tqdm(ticker_list):
    print('Processing: %s' %ticker)
    
    # Skip counters which have already been downloaded
    if '%s.csv' %ticker in os.listdir(DOWNLOAD_FOLDER):
        continue 
    
    url= 'https://finance.yahoo.com/quote/%s/history?' \
         'period1=%s&period2=%s&interval=1d&filter=history&frequency=1d' %(ticker, time_str1, time_str2)
    print(url)
    
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(url)
    
    dload_button_xpath = '//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[2]/span[2]/a'
    element_present = EC.presence_of_element_located(
                (By.XPATH, dload_button_xpath))
    try:
        WebDriverWait(driver, 10).until(element_present)
    except:
        print(ticker)
        wrong_ticker_list.append([ticker, "Driver Wait too long"])
        continue
        
    dload_button = driver.find_element_by_xpath(dload_button_xpath)
    dload_button.click()
    
    try:
        error_msg = driver.find_element_by_xpath('/html/body/pre').text
        print('Ticker Error: %s, %s' %(ticker, error_msg))
        wrong_ticker_list.append([ticker, error_msg])
        continue
    except Exception as e:
        pass
        
    # Wait for download to complete by checking for csv file locally
    # Note that repeated files not downloaded
    while ('%s.csv' %ticker not in os.listdir(DOWNLOAD_FOLDER)):
        time.sleep(0.1)
    driver.close()
wrong_ticker_list

In [ ]:
wrong_ticker_list

- To run faster, restart the kernel and run again
- Sometime might get this error, just retry and it should be okay
    - WebDriverException: Message: unknown error: unable to discover open pages (FIXED)
    - SessionNotCreatedException: Message: session not created from tab crashed (Session info: headless chrome=84.0.4147.135)

- Error examples for wrong tickers
    - 404 Not Found: No data found, symbol may be delisted
    - 404 Not Found: Timestamp data missing
    - 400 Bad Request: Data doesn't exist for startDate = 1546272000, endDate = 1577808000

- Sometimes a valid symbol may not retrieve the data (if is in the bad_ticker list, with label "Driver Wait too long", but just rerun the code above and it should dload the data properly

- Bad Ticker Data (from above)
    - [['BRK.B', '404 Not Found: No data found, symbol may be delisted'],
    - ['BF.B', '404 Not Found: Timestamp data missing.'],
    - ['CARR', "400 Bad Request: Data doesn't exist for startDate = 1546272000, endDate = 1577808000"],
    - ['OTIS', "400 Bad Request: Data doesn't exist for startDate = 1546272000, endDate = 1577808000"]]

In [ ]:
# Concat all dfs to form list of all prices
list_of_dfs = []
for filename in os.listdir(DOWNLOAD_FOLDER):
    df = pd.read_csv(DOWNLOAD_FOLDER + filename, index_col=0)[['Adj Close']]
    ticker = filename.split('.')[0]
    df.rename({'Adj Close':ticker}, axis=1, inplace=True)
    list_of_dfs.append(df)
ticker_price_df = pd.concat(list_of_dfs, axis=1)
ticker_price_df.to_csv('data_out/russell_price_df.csv')
ticker_price_df

In [ ]:
# OPTIONAL: Remove all downloaded files

# for filename in os.listdir(DOWNLOAD_FOLDER):
#     os.remove(DOWNLOAD_FOLDER + filename)

##### Price Scraping (Method 2: By scrolling down the page and scraping all values)

- Method 1 is better
- will take longer for longer pages 
- difficult to scrape dates, as some dates are for dividends (not scraped in this case)
- other than dates, output should be the same as Method 1


In [ ]:
ticker_list = tickers_df.Ticker[:2]
all_prices_dict = {}

date1 = datetime.strptime('20190101', "%Y%m%d")
date2 = datetime.strptime('20200101', "%Y%m%d")

# Yahoo webpage only shows 100 prices at a time
NUM_TIMES_TO_SCROLL = (date2 - date1).days//100 + 1

time_str1 = str(int(datetime.timestamp(date1)))
time_str2 = str(int(datetime.timestamp(date2)))

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--no-sandbox') # Bypass OS security model
options.add_argument('--headless')

for ticker in tqdm(ticker_list):
    print('Processing: %s' %ticker)
    
    url= 'https://finance.yahoo.com/quote/%s/history?' \
         'period1=%s&period2=%s&interval=1d&filter=history&frequency=1d' %(ticker, time_str1, time_str2)
    print(url)
    
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(url)

    SCROLL_PAUSE_TIME = 0.1

    # getting scrollHeight using javascript dosent work for YahooFinance pg
    for _ in range(NUM_TIMES_TO_SCROLL):
        driver.execute_script("window.scrollTo(0, 100000);")
        time.sleep(SCROLL_PAUSE_TIME)
    
    items = driver.find_elements_by_xpath(
        '//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[2]/table/tbody/tr/td[6]/span')
    prices = list(reversed(list(map(lambda x: x.text, items))))
    all_prices_dict[ticker] = prices
    driver.close()
print(pd.DataFrame(all_prices_dict))
ticker_price_df2 = pd.DataFrame(all_prices_dict)
ticker_price_df2.to_csv('data_out/ticker_price_df2.csv')
display(ticker_price_df2)

In [ ]:
filter_prices_dict = {k:v for k,v in all_prices_dict.items() if len(v)==253}
ticker_price_df2 = pd.DataFrame(filter_prices_dict)
ticker_price_df2.to_csv('data_out/snp_price_scrolling_df.csv')
display(ticker_price_df2)

In [ ]:
# Tickers with errors (incomplete data)
{k:len(v) for k,v in all_prices_dict.items() if len(v)!=253}

- Runtime
    - 2:45:42 (505/505 [2:45:42<00:00, 19.69s/it]

In [ ]:
from playsound import playsound
def ALARM():
    for i in range(10):
        playsound('data_in/bell.mp3')

ALARM()

##### Ratio Scraping 

In [ ]:
ticker_list = tickers_df.Ticker[:2]
# ticker_list = ['BRK.B']
ticker_ratios_df = pd.DataFrame(columns = ['Ticker', 'mkt_cap', 'pb_ratio', 'beta', 'profit_margin', 'ROA', 'ROE'])
wrong_ticker_ratio_list = []

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--headless')
options.add_argument('--no-sandbox') # Bypass OS security model

for ticker in tqdm(ticker_list):
    url = 'https://finance.yahoo.com/quote/'+ticker+'/key-statistics?p='+ticker
    print(url)    
    
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    
    mkt_cap_xpath = '//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[3]/div[1]/div[2]/div/div[1]/div[1]/table/tbody/tr[1]/td[3]'
    element_present = EC.presence_of_element_located(
                (By.XPATH, mkt_cap_xpath))
    try:
        WebDriverWait(driver, 10).until(element_present)
    except:
        print(ticker)
        wrong_ticker_ratio_list.append(ticker)
        continue
        
    mkt_cap = driver.find_element_by_xpath('//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[3]/div[1]/div[2]/div/div[1]/div[1]/table/tbody/tr[1]/td[3]').text
    pb_ratio = driver.find_element_by_xpath('//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[3]/div[1]/div[2]/div/div[1]/div[1]/table/tbody/tr[7]/td[3]').text
    beta = driver.find_element_by_xpath('//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[3]/div[2]/div/div[1]/div/div/table/tbody/tr[1]/td[2]').text
    profit_margin = driver.find_element_by_xpath('//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[3]/div[3]/div/div[2]/div/div/table/tbody/tr[1]/td[2]').text
    roa = driver.find_element_by_xpath('//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[3]/div[3]/div/div[3]/div/div/table/tbody/tr[1]/td[2]').text
    roe = driver.find_element_by_xpath('//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[3]/div[3]/div/div[3]/div/div/table/tbody/tr[2]/td[2]').text

    ticker_ratios_df.loc[len(ticker_ratios_df)] = [ticker, mkt_cap, pb_ratio, beta, profit_margin, roa, roe]
    driver.close()
display(ticker_ratios_df)
wrong_ticker_ratio_list

In [ ]:
def percentage_to_float(s):
    if s =='N/A':
        return np.NaN
    return float(s.strip('%').replace(',',''))/100

def mkt_cap_converter(s):
    multiplier = s[-1]
    if multiplier == 'B':
        return float(s[:-1]) # * 1000000000
    if multiplier == 'T':
        return float(s[:-1]) * 1000
    else:
        raise Exception(s)
        
def pb_ratio_converter(s):
    if s == 'N/A':
        return np.NaN
    multiplier = s[-1]
    if multiplier == 'k':
        return float(s[:-1]) * 1000
    else:
        return float(s)

In [ ]:
ticker_ratios_clean_df = ticker_ratios_df.copy()
ticker_ratios_clean_df.ROA = ticker_ratios_df.ROA.apply(percentage_to_float)
ticker_ratios_clean_df.ROE = ticker_ratios_df.ROE.apply(percentage_to_float)
ticker_ratios_clean_df.profit_margin = ticker_ratios_df.profit_margin.apply(percentage_to_float)
ticker_ratios_clean_df.mkt_cap = ticker_ratios_df.mkt_cap.apply(mkt_cap_converter)
ticker_ratios_clean_df.pb_ratio = ticker_ratios_df.pb_ratio.apply(pb_ratio_converter)
ticker_ratios_clean_df.beta = ticker_ratios_clean_df.beta.replace('N/A', np.NaN).astype(float)

ticker_ratios_clean_df

In [ ]:
ticker_ratios_clean_df.to_csv('data_out/russell_ratios_df.csv', index=False)

# 10K reports merging

- Set path where 10K reports .txt files are and merge them into a dataframe, and save accordingly

In [3]:
DOWNLOAD_FOLDER_10K = os.getcwd() + os.path.sep + 'data_in\\10K intros\\russell15\\'
desc_list = []

for filename in tqdm(os.listdir(DOWNLOAD_FOLDER_10K)):
    if filename == 'desktop.ini': # not sure why its there, but cant seem to be able to take it out
        continue
    ticker,cik,report,date,_,_ = filename.split('_')
    with open(DOWNLOAD_FOLDER_10K + filename,  'r', encoding='utf-8') as f:
        desc = f.read()
        desc = desc.strip().replace('\n', ' ').replace(',', ' ')
    desc_list.append([ticker,cik,desc,report,date])

desc_10K_df = pd.DataFrame(desc_list, columns=['Ticker','CIK', 'Description', 'Report', 'Date'])

desc_10K_df.sort_values(by=['Ticker', 'Date'], inplace=True)
desc_10K_df.drop_duplicates('Ticker', keep='last', inplace=True)
desc_10K_df = desc_10K_df[desc_10K_df.Report == '10K'][['Ticker','CIK', 'Description']]
desc_10K_df.reset_index(drop=True, inplace=True)

desc_10K_df

,Ticker,CIK,Description
0,A,0001090872,PART I Item 1. Business Overview Agilent T...
1,AAL,0000004515,PART I ITEM 1. BUSINESS Overview Ame...
2,AAN,0000706688,PART I. ITEM 1. BUSINESS Unless otherwise in...
3,AAOI,0001158114,PART I Item 1. Business Forward-Looking In...
4,AAON,0000824142,PART I Item 1. Business. General Development...
...,...,...,...
2241,ZIXI,0000855612,PART I Item 1. Business Zix Corporation...
2242,ZNGA,0001439404,PART I ITEM 1. BUSINESS Overview Zyng...
2243,ZTS,0001555280,PART I Item 1. Business. Overview Zoetis I...
2244,ZUMZ,0001318008,Item 1. BUSINESS Zumiez is a leading multi-...


In [5]:
desc_10K_df.to_pickle('data_out/russell_desc_10K2015.pkl')

# GICS code to Industry mapping scraping

In [ ]:
gics_code_to_industry_df = pd.read_html('https://en.wikipedia.org/wiki/Global_Industry_Classification_Standard')[0][['Industry', 'Industry.1']].drop_duplicates()
gics_code_to_industry_df.columns = ['GICS', 'GICS Industry']
gics_code_to_industry_df.reset_index(inplace=True, drop=True)
gics_code_to_industry_df

In [ ]:
gics_code_to_industry_df.shape

In [ ]:
gics_code_to_industry_df.to_csv('data_out/gics_code_to_industry.csv', index=False)

# Clean GICS classification

- Get Ticker to GICS mapping
- The GICS hierarchy begins with 11 sectors and is followed by 24 industry groups, 68 industries, and 157 sub-industries.
- **Not important bcos found better data source in data_in**

In [ ]:
gics_df = pd.read_csv('data_in/snp_GICS_classification_bloomberg.csv', names=['Ticker', 'GICS', 'NAICS'], dtype=str)

In [ ]:
gics_filter_df = gics_df[~gics_df.GICS.isna()].reset_index(drop=True).iloc[:,:-1]

In [ ]:
len(gics_filter_df.GICS.value_counts())

In [ ]:
gics_filter_df['GICS'] = gics_filter_df.GICS.apply(lambda s: s[:6])
gics_filter_df

In [ ]:
len(gics_filter_df.GICS.value_counts())

In [ ]:
gics_filter_df.to_csv('data_out/ticker_to_gics_bloomberg.csv', index=False)

# List of CIKs for relevant tickers

- currently just collect all CIKS for all the tickers from output from the above section (data_out/ticker_to_gics.csv)
- website contains 10102 tickers-cik mappings
- need this to scrape 10K reports

In [ ]:
import urllib.request

url = "https://www.sec.gov/include/ticker.txt"
html = urllib.request.urlopen(url).read()
soup = bs.BeautifulSoup(html)
html_text = soup.find('p').text
ticker_to_cik_list = [cpy.upper().split('\t') for cpy in html_text.split('\n')]
ticker_to_cik_df = pd.DataFrame(ticker_to_cik_list, columns=['Ticker','CIK'])
ticker_to_cik_df

In [ ]:
ticker_to_gics_df = pd.read_csv('data_in/ticker_to_gics.csv', names=['Ticker', 'GICS'])
ticker_to_gics_df.Ticker = ticker_to_gics_df.Ticker.str.replace('/','-')
ticker_to_gics_df

In [ ]:
#Test
display(snp_tickers_df[~snp_tickers_df.Ticker.isin(ticker_to_gics_df.Ticker)])
display(russell_tickers_df[~russell_tickers_df.Ticker.isin(ticker_to_gics_df.Ticker)])

In [ ]:
scrapping_ticker_ciks_df = pd.merge(ticker_to_gics_df, ticker_to_cik_df, on='Ticker', how='inner')[['CIK', 'Ticker']]
scrapping_ticker_ciks_df

In [ ]:
scrapping_ticker_ciks_df.to_csv('data_out/scrapping_ticker_ciks.txt', sep=" ", index=False, header=False)

# Scrape Wikipedia data

In [33]:
def get_wiki_details(cpy):
    return_for_invalid_entry = [None]
    if pd.isna(cpy):
        return return_for_invalid_entry
    try:
        p = wiki.page('%s company'%cpy) # Can also do wiki.summary('APPL company')
        title = p.title
        title_list = re.sub('[^a-zA-Z0-9]', ' ', title).lower().split()
        cpy_list = re.sub('[^a-zA-Z0-9]', ' ', cpy).lower().split()
        common_words_list = [word for word in cpy_list if word in title_list]
        if not common_words_list:
            summary = p.summary
            summary_list = re.sub('[^a-zA-Z0-9]', ' ', summary).lower().split()
            if not [word for word in cpy_list if word in summary_list]:
                print('wrong search: %s (search title: %s)'%(cpy, title))
                return return_for_invalid_entry
    except:
        print('no wiki result: %s'%cpy)
        return return_for_invalid_entry
    desc = p.summary.strip().replace('\n', ' ')
    return [desc] 

In [34]:
# Test
# p = wiki.page('Equinix company')
# p.summary

In [35]:
wiki_tickers_df = russell_tickers_df
wiki_desc_list = [get_wiki_details(cpy) for cpy in tqdm(wiki_tickers_df.Name)]

no wiki result: Aaron's Holdings Company, Inc.
wrong search: Applied Optoelectronics, Inc. (search title: List of flat panel display manufacturers)
no wiki result: Abeona Therapeutics Inc.
no wiki result: Allegiance Bancshares, Inc.
no wiki result: Arcosa, Inc.
wrong search: Atlantic Capital Bancshares, Inc. (search title: Truist Financial)
wrong search: Acacia Communications, Inc. (search title: List of acquisitions by Cisco Systems)
no wiki result: ACNB Corporation
no wiki result: Ares Commercial Real Estate Corporation
no wiki result: Aclaris Therapeutics, Inc.
no wiki result: AcelRx Pharmaceuticals, Inc.
no wiki result: Agree Realty Corporation
no wiki result: ADMA Biologics, Inc.
no wiki result: Adamas Pharmaceuticals, Inc.
no wiki result: Aduro Biotech, Inc.
no wiki result: Addus HomeCare Corporation
no wiki result: Affimed N.V.
no wiki result: Aeglea BioTherapeutics, Inc.
no wiki result: AGNC Investment Corp.
no wiki result: PlayAGS, Inc.
no wiki result: Argan, Inc.
wrong search

c:\users\sival\appdata\local\programs\python\python36\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\users\sival\appdata\local\programs\python\python36\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


no wiki result: Cadiz Inc.
no wiki result: Celcuity Inc.
no wiki result: Celsius Holdings, Inc.
no wiki result: Cerecor Inc.
no wiki result: Cerus Corporation
no wiki result: CrossFirst Bankshares, Inc.
wrong search: Conformis, Inc. (search title: 2020 in Japanese music)
no wiki result: Chemed Corporation
wrong search: Chiasma, Inc. (search title: List of College of the Holy Cross alumni)
wrong search: Chemung Financial Corporation (search title: Chester Adgate Congdon)
no wiki result: Charah Solutions, Inc.
no wiki result: Coherus BioSciences, Inc.
wrong search: Chimera Investment Corporation (search title: Russell 1000 Index)
no wiki result: Civista Bancshares, Inc.
wrong search: CompX International Inc. (search title: NL Industries)
no wiki result: Clarus Corporation
no wiki result: Chatham Lodging Trust
no wiki result: Clean Energy Fuels Corp.
wrong search: Clipper Realty Inc. (search title: List of Jewish American businesspeople in real estate)
no wiki result: Calyxt, Inc.
wrong s

no wiki result: Getty Realty Corp.
wrong search: GTY Technology Holdings Inc. (search title: List of S&P 600 companies)
wrong search: GWG Holdings, Inc. (search title: 2002 Stanley Cup Finals)
no wiki result: Harpoon Therapeutics, Inc.
no wiki result: Hannon Armstrong Sustainable Infrastructure Capital, Inc.
wrong search: Horizon Bancorp, Inc. (search title: Chemical Bank)
no wiki result: Heritage-Crystal Clean, Inc
wrong search: HCI Group, Inc. (search title: Nielsen Holdings)
no wiki result: Highwoods Properties, Inc.
no wiki result: Hamilton Lane Incorporated
no wiki result: Hallador Energy Company
wrong search: Hooker Furniture Corporation (search title: Hicksville, New York)
wrong search: HarborOne Bancorp, Inc. (search title: National Bank of Detroit)
wrong search: HOOKIPA Pharma Inc. (search title: Thomas P. Monath)
no wiki result: HighPoint Resources Corporation
wrong search: Healthcare Realty Trust Incorporated (search title: List of S&P 400 companies)
no wiki result: Herc Hol

no wiki result: OptiNose, Inc.
wrong search: Organogenesis Holdings Inc. (search title: Fluoxetine)
no wiki result: Osmotica Pharmaceuticals plc
wrong search: OraSure Technologies, Inc. (search title: List of S&P 600 companies)
no wiki result: Outfront Media Inc. (REIT)
no wiki result: Ohio Valley Banc Corp.
wrong search: Oyster Point Pharma, Inc. (search title: Halsey (singer))
no wiki result: Phibro Animal Health Corporation
no wiki result: PaySign, Inc.
wrong search: PCB Bancorp (search title: First Community Bancshares)
no wiki result: Pacira BioSciences, Inc.
no wiki result: PCSB Financial Corporation
no wiki result: Pure Cycle Corporation
wrong search: PDC Energy, Inc. (search title: Baker Hughes)
no wiki result: PDL Community Bancorp
wrong search: Piedmont Office Realty Trust, Inc. (search title: Aon Center (Chicago))
wrong search: Parsley Energy, Inc. (search title: Pioneer Natural Resources)
no wiki result: Pebblebrook Hotel Trust
no wiki result: PetIQ, Inc.
no wiki result: Pe

wrong search: TowneBank (search title: Yale (company))
wrong search: TPI Composites, Inc. (search title: Pearson Yachts)
no wiki result: Trecora Resources
wrong search: Tabula Rasa HealthCare, Inc. (search title: Radius Ventures)
no wiki result: Terreno Realty Corporation
no wiki result: Transcat, Inc.
wrong search: Tronox Holdings plc (search title: Corporation tax in the Republic of Ireland)
no wiki result: TriMas Corporation
wrong search: TPG RE Finance Trust, Inc. (search title: Dell)
wrong search: TransEnterix, Inc. (search title: VxWorks)
no wiki result: Timberland Bancorp, Inc.
no wiki result: TriState Capital Holdings, Inc.
wrong search: TTM Technologies, Inc. (search title: List of S&P 600 companies)
wrong search: Twist Bioscience Corporation (search title: List of companies based in Austin, Texas)
no wiki result: TherapeuticsMD, Inc.
no wiki result: Tyme Technologies, Inc.
no wiki result: Unifi, Inc.
no wiki result: AMERCO
no wiki result: Universal Health Realty Income Trust


snp: 

    wrong search: E*Trade (search title: Expotrade Arena)
    wrong search: Equinix (search title: Equinox)
    no wiki result: Laboratory Corp. of America Holding
    no wiki result: Linde plc
    no wiki result: Zoetis

In [36]:
desc_wiki_df = pd.concat([wiki_tickers_df[['Ticker']], pd.DataFrame(wiki_desc_list, columns=['Description'])], axis=1)
desc_wiki_df.head(100)

,Ticker,Description
0,A,"Agilent Technologies, Inc. is a global analyti..."
1,AA,Alcoa Corporation (a portmanteau of Aluminum C...
2,AAL,American Airlines Group Inc. is an American pu...
3,AAN,None
4,AAOI,None
5,AAON,"AAON Inc. (NASDAQ: AAON) designs, manufactures..."
6,AAP,"Advance Auto Parts, Inc. (Advance) is an Ameri..."
7,AAPL,Apple Inc. is an American multinational techno...
8,AAT,Alphabet Inc. is an American multinational con...
9,AAWW,"Atlas Air, Inc., a wholly owned subsidiary of ..."


In [37]:
desc_wiki_df.to_csv('data_out/russell_desc_wiki2.csv', index=False)

# Clean Russell Ratios

In [38]:
def percentage_to_float(s):
    if type(s) == float: #if np.nan
        return s
    return float(s.strip('%').replace(',',''))/100

def mkt_cap_converter(s):
    '''
    output: mkt_cap (in million)
    '''
    if type(s) == float: #if np.nan
        return s
    multiplier = s[-1]
    if multiplier == 'M':
        return float(s[:-1])
    if multiplier == 'B':
        return float(s[:-1]) * 1000
    if multiplier == 'T':
        return float(s[:-1]) * 1000000
    else:
        raise Exception(s)
        
def pb_ratio_converter(s):
    if type(s) == float: #if np.nan
        return s
    multiplier = s[-1]
    if multiplier == 'k':
        return float(s[:-1]) * 1000
    else:
        return float(s)
    
def remove_commas(s):
    if type(s) == float: #if np.nan
        return s
    return float(s.replace(',',''))

In [39]:
ticker_ratios_df = pd.read_csv('data_in/russell_ratios.csv')
ticker_ratios_df.columns = ['Ticker', 'mkt_cap', 'pb_ratio', 'beta', 'profit_margin', 'ROA', 'ROE']

ticker_ratios_df.mkt_cap = ticker_ratios_df.mkt_cap.apply(mkt_cap_converter)
ticker_ratios_df.pb_ratio = ticker_ratios_df.pb_ratio.apply(pb_ratio_converter)
ticker_ratios_df.beta = ticker_ratios_df.beta.apply(remove_commas)
ticker_ratios_df.profit_margin = ticker_ratios_df.profit_margin.apply(percentage_to_float)
ticker_ratios_df.ROA = ticker_ratios_df.ROA.apply(percentage_to_float)
ticker_ratios_df.ROE = ticker_ratios_df.ROE.apply(percentage_to_float)

ticker_ratios_df

,Ticker,mkt_cap,pb_ratio,beta,profit_margin,ROA,ROE
0,MMM,78520.00,7.80,1.00,0.1537,0.1040,0.4992
1,ABT,139580.00,4.49,0.97,0.1115,0.0459,0.1156
2,ABBV,112510.00,NaN,0.78,0.2477,0.1180,NaN
3,ABMD,6530.00,6.24,0.86,0.2414,0.1372,0.2028
4,ACN,114850.00,7.59,1.03,0.1113,0.1344,0.3401
...,...,...,...,...,...,...,...
2886,PMT,1060.00,0.49,1.09,0.0000,-0.0433,-0.2376
2887,COLL,560.22,6.41,1.07,-0.0421,-0.0117,-0.1106
2888,FLIC,412.81,1.06,0.52,0.3688,0.0096,0.1045
2889,GWR,6300.00,1.81,1.31,0.0928,0.0308,0.0580


In [40]:
ticker_ratios_df.to_csv('data_out/russell_ratios_clean.csv', index=False)

# Extra

##### Implicit wait, selenium (google.com)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome()
# driver.implicitly_wait(10)

driver.get("http://google.com")
driver.maximize_window()

print("Implicit Wait Example")

inputElement = driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div[1]/div[1]/div/div[2]/input')  # .find_element_by_id("lst-ib")
inputElement.send_keys("Techbeamers")
inputElement.submit()

driver.close()

##### Multiprocessing test

- for pool to work on jupyter notebook need to import function
- with muti processing cannot change global variables (there is a fix for this)
- printing is not trivial in multiprocessing

- still need to figure out if multiprocessing or multithreading is more suitable for scraping

In [ ]:
%%timeit
p=Pool(processes = 4)
output = p.map(multithreading_test.worker,range(3000000))
print(output[-3:])

In [ ]:
%%timeit
p = Pool(processes=4)
results = p.map_async(multithreading_test.worker, range(3000000))
output = results.get()
print(output[-3:])

In [ ]:
%%timeit
output = list(map(multithreading_test.worker, range(3000000)))
print(output[-3:])

- I think in this case using ordinary functions is faster than using Pool due to the high over head
- only use pool when there is high CPU requirement, high iterations

In [ ]:
# Number of cores for multiprocessing
import multiprocessing
multiprocessing.cpu_count()

##### Check Chrome Driver Version

In [ ]:
driver = webdriver.Chrome()
str1 = driver.capabilities['browserVersion']
str2 = driver.capabilities['chrome']['chromedriverVersion'].split(' ')[0]
print(str1)
print(str2)
print(str1[0:2])
print(str2[0:2])
if str1[0:2] != str2[0:2]: 
    print("please download correct chromedriver version")

##### Running other scripts

In [ ]:
%run 

##### Yield

In [ ]:
def f():
    yield 1
    yield 2
    yield 3
[i for i in f()]

In [ ]:
print(f())

##### Scrapy Tutorial

In [ ]:
# Settings for notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess

In [ ]:
import json

class JsonWriterPipeline(object):
    def open_spider(self, spider):
        self.file = open('quoteresult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [ ]:
import logging

class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
        'http://quotes.toscrape.com/page/2/',
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'quoteresult.json'                        # Used for pipeline 2
    }
    
    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract_first(),
                'author': quote.css('span small::text').extract_first(),
                'tags': quote.css('div.tags a.tag::text').extract(),
            }

In [ ]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(QuotesSpider)
process.start()

In [ ]:
import pandas as pd
dfjson = pd.read_json('quoteresult.json')
dfjson

In [ ]:
dfjl = pd.read_json('quoteresult.jl', lines=True)
dfjl

In [ ]:
dfjson.to_pickle('quotejson.pickle')
dfjl.to_pickle('quotejl.pickle')

##### tqdm

In [ ]:
from tqdm import tqdm

j=0
for i in tqdm(range(1000000), ):
    j+=i
    
print(j)

In [ ]:
import time
import sys
from tqdm import trange


def do_something():
    time.sleep(1)

def do_another_something():
    time.sleep(1)


for i in trange(10):
    do_something()

#     for j in trange(2):
#         do_another_something()

In [ ]:
from tqdm.notebook import trange, tqdm
import time

for i in trange(6):
    do_something()

    for j in trange(2):
        do_another_something()


##### Test Scraping 10K reports

In [ ]:

for file in os.listdir("data_in"):
    if file.endswith(".txt") and "Item1_excerpt" in file:
        print(file)
        with open("data_in/" + file, "rt",encoding='utf-8') as f:
            line_num = 1
            for line in f:
                line = line.lower()
                if ("part i" in line) or ("item 1" in line) or ("business" in line and len(line) < 20) or ("introduction" in line):
                    print(line_num, line)                    
                if ("item 1a" in line) or ("risk factors" in line):
                    print(line_num, line)                    
                line_num += 1
        break

In [ ]:
import glob

path = 'data_in'

files = [f for f in glob.glob(path + "*/*.txt", recursive=True)]

for f in files:
    print(f)

In [ ]:
glob.glob(path + "*/*.txt", recursive=True)

###### Test if df contains .

In [ ]:
snp_tickers_df[snp_tickers_df.Ticker.str.contains(".", regex=False)]